<a href="https://colab.research.google.com/github/chicknSarah/CAC_GreenAI/blob/main/CAC_GreenAI_forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The first block of code underneath each energy region updates our data base (the files formatted like "US-CAL-CISO") with the past 24 hours of data from electricitymaps API. It writes yesterday's CI onto sheets like "yesterdayCAL". This keeps our forecast relevant and more accurate. The second block of code forecasts the next 24 hours of CI and writes the data onto files like "forecastCAL", writes the 4 lowest CI hours onto sheets like "lowsCAL", and writes the avg CI onto sheets like "avgCAL".

All of this data is read by Thunkable which is connected to the sheets of all the energy regions, and is used to display charts, forecasts, calculate the estimated amount of CO2 you saved if you followed the forecast, and provide tips.

Examples of California's sheets (to test, make a copy of the google sheets and name them accordingly):

US-CAL-CISO: https://docs.google.com/spreadsheets/d/1fXWmgplxZnury6xz_Pc6s4s-RMSkD87LruSk9R9A3uQ/edit?usp=sharing

yesterdayCAL: https://docs.google.com/spreadsheets/d/18cZgL2rjY-2LZSCyp7NiwEbjtD63xBDwWTOv9BFcdbk/edit?usp=sharing

forecastCAL: https://docs.google.com/spreadsheets/d/1GgC2fS3eyFFpNhNaGo4p2vLSgz87N4jZyI_5MKRKmnc/edit?usp=sharing

avgCAL: https://docs.google.com/spreadsheets/d/1Rg_DQR7SsH_Txcuy7JDjUk67B27KgKMA52aKnKUeGm4/edit?usp=sharing

lowsCAL: https://docs.google.com/spreadsheets/d/1ScrFetNHhYqPdlMR5vlO0qfQ0CR2DGxy-sjsucq1QSI/edit?usp=sharing

CHANGE TOKEN TO YOUR API KEY

In [ ]:
# Daily update from here
# Authenticate and create a client
from google.colab import auth
auth.authenticate_user()
from google.auth import default
import gspread

creds, _ = default()
gc = gspread.authorize(creds)

////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
                                    
                                    US-CAL-CISO below

////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

In [ ]:
# CAL-CISO  updating past data

import pandas as pd
from prophet import Prophet
import gspread
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.auth import default
import requests

# Authenticate and create a client
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

token = "YOUR API KEY"
# API call: past 24 hours of carbon intensity data of energy region
response = requests.get(
    "https://api.electricitymap.org/v3/carbon-intensity/history?zone=US-CAL-CISO",
    headers={
        "auth-token": token
    }
)


if response.status_code == 200:
    # If the request was successful, parse the data
    global histCI
    data = response.json()
    histCI = pd.DataFrame.from_dict(data['history'])

else:
    print(f'Failed to retrieve data: {response.status_code}')

update = gc.open("US-CAL-CISO").sheet1
n = update.get_all_values()
new = pd.DataFrame(n, columns=["ds","y"])
z = pd.to_datetime(histCI["datetime"])
df = pd.DataFrame(z, columns=["datetime"])
df["new"]=df["datetime"].astype(str)
df["Carbon"] = histCI["carbonIntensity"]
n=df.drop("datetime", axis="columns")
n.rename(columns={"new":"ds"}, inplace=True)

def write_df_to_sheets(df, sheet_name):
  values = [df.columns.values.tolist()] + df.values.tolist()
  sheet = gc.open(sheet_name).sheet1
  sheet.update(values)

l=[]
t=[]
#reformatting data from electricity maps and updating the lists
p = n["ds"]
data = pd.DataFrame(p)
for i in range (24):
  s = str(data.loc[i,"ds"])
  formatted = s.replace("+00:00","")
  clean = formatted.split(" ")
  time = clean[1]
  final = time.split(":")
  final1 = final[0]
  l.append(final1)
  t.append(formatted)
c = n["Carbon"]
data2 = {
    "ds" : l,
    "y" : c
}

unformatted = {
    "ds" : t,
    "y" : c
}

df2 = pd.DataFrame(data2)

unformattedDF = pd.DataFrame(unformatted)

newdf = new._append(unformattedDF, ignore_index=False)

# appending the electricity maps data onto the region's datasheet
write_df_to_sheets(newdf, "US-CAL-CISO")

lows = df.sort_values(by="Carbon", ascending=True)

data = pd.DataFrame()

write_df_to_sheets(df2, "yesterdayCAL")

In [ ]:
# US-CAL-CISO  forecasting
region = "US-CAL-CISO"
starting = 18352

import pandas as pd
from prophet import Prophet
import gspread
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.auth import default
from datetime import date
from datetime import datetime

def numOfDays(date1, date2):
  #check which date is greater to avoid days output in negative number
    if date2 > date1:
        return (date2-date1).days
    else:
        return (date1-date2).days

# Driver program
created = date(2025, 10, 25)
d = datetime.now().date()
diff = int(numOfDays(created, d))
gap = 24*diff

# Authenticate and create a client
creds, _ = default()
gc = gspread.authorize(creds)
auth.authenticate_user()

#reads the region's CI history
data = gc.open(region).sheet1
d = data.get_all_values()
data = pd.DataFrame(d, columns=["ds","y"])
df=data.drop(index=0)

#fits the ML model on the data
m = Prophet(changepoint_prior_scale=0.01).fit(df)
future = m.make_future_dataframe(periods=24, freq='H')
fcst = m.predict(future)

# isolates the last 24 values which represents the next 24 hrs of CI forecast
x24 = fcst.tail(24)
ascending = x24.sort_values(by=["yhat"], ascending=True)
filter = ["ds","yhat"]
almost=ascending[filter]
almost["string"] = almost["ds"].astype("string")

fil = ["string", "yhat"]
final = almost[fil]
ok = final.head(4)
sum=0
print(x24)

dfx_listY = []
dfx_listDS = []

# calculates avg CI for the day
for i in range(24):
  y=i+starting+gap
  x=x24.loc[y,"yhat"]
  dfx_listY.append(x)
  ds=x24.loc[y,"ds"]
  dfx_listDS.append(str(ds))
  sum+=x

avg = sum/24
print(avg)

dfx = pd.DataFrame({"ds":dfx_listDS, "y":dfx_listY})

def write_df_to_sheets(df, sheet_name):
  values = [df.columns.values.tolist()] + df.values.tolist()
  sheet = gc.open(sheet_name).sheet1
  sheet.update(values)

# writes the forecast to a google sheet
write_df_to_sheets(dfx, "forecastCAL")

avDF = pd.DataFrame({"yhat":[avg]})

#writes the 4 low CI hours of tomorrow onto a google sheet, as well as the average CI
write_df_to_sheets(ok, "lowsCAL")
write_df_to_sheets(avDF, "avgCAL")

DEBUG:cmdstanpy:input tempfile: /tmp/tmpg56yps50/b7l4fpii.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpg56yps50/v4spqkxy.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.12/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=97562', 'data', 'file=/tmp/tmpg56yps50/b7l4fpii.json', 'init=/tmp/tmpg56yps50/v4spqkxy.json', 'output', 'file=/tmp/tmpg56yps50/prophet_modeldsxd9ukd/prophet_model-20251025010831.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
01:08:31 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
01:08:35 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
/usr/local/lib/python3.12/dist-packages/prophet/forecaster.py:1872: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  dates = pd.date_range(
/tmp/ipython-input-415328387.py:48: SettingWithCopy

                       ds       trend  yhat_lower  yhat_upper  trend_lower  \
18352 2025-10-25 02:00:00  191.053007  228.342643  297.062313   191.053007   
18353 2025-10-25 03:00:00  191.052724  233.833932  302.336133   191.052724   
18354 2025-10-25 04:00:00  191.052442  232.760442  303.270324   191.052442   
18355 2025-10-25 05:00:00  191.052160  234.892839  301.481825   191.052160   
18356 2025-10-25 06:00:00  191.051877  236.296262  305.633590   191.051877   
18357 2025-10-25 07:00:00  191.051595  236.373462  304.037120   191.051595   
18358 2025-10-25 08:00:00  191.051313  231.779073  302.239720   191.051313   
18359 2025-10-25 09:00:00  191.051031  230.518731  299.398308   191.051031   
18360 2025-10-25 10:00:00  191.050748  226.506224  298.300459   191.050748   
18361 2025-10-25 11:00:00  191.050466  228.931689  302.137562   191.050466   
18362 2025-10-25 12:00:00  191.050184  230.155833  300.031071   191.050184   
18363 2025-10-25 13:00:00  191.049901  219.573972  291.303349   

////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
                                    
                                    US-NY-NYIS below

////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

In [ ]:
# US-NY-NYIS  updating past data

import pandas as pd
from prophet import Prophet
import gspread
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.auth import default
import requests

# Authenticate and create a client
creds, _ = default()
gc = gspread.authorize(creds)

# API call: past 24 hours of carbon intensity data of energy region
response = requests.get(
    "https://api.electricitymap.org/v3/carbon-intensity/history?zone=US-NY-NYIS",
    headers={
        "auth-token": token
    }
)


if response.status_code == 200:
    # If the request was successful, parse the data
    global histCI
    data = response.json()
    histCI = pd.DataFrame.from_dict(data['history'])
else:
    print(f'Failed to retrieve data: {response.status_code}')

update = gc.open("US-NY-NYIS").sheet1
n = update.get_all_values()
new = pd.DataFrame(n, columns=["ds","y"])
z = pd.to_datetime(histCI["datetime"])
df = pd.DataFrame(z, columns=["datetime"])
df["new"]=df["datetime"].astype(str)
df["Carbon"] = histCI["carbonIntensity"]
n=df.drop("datetime", axis="columns")
n.rename(columns={"new":"ds"}, inplace=True)

def write_df_to_sheets(df, sheet_name):
  values = [df.columns.values.tolist()] + df.values.tolist()
  sheet = gc.open(sheet_name).sheet1
  sheet.update(values)

l=[]
t=[]
#reformatting data from electricity maps and updating the lists
p = n["ds"]
data = pd.DataFrame(p)
for i in range (24):
  s = str(data.loc[i,"ds"])
  formatted = s.replace("+00:00","")
  clean = formatted.split(" ")
  time = clean[1]
  final = time.split(":")
  final1 = final[0]
  l.append(final1)
  t.append(formatted)
c = n["Carbon"]
data2 = {
    "ds" : l,
    "y" : c
}

unformatted = {
    "ds" : t,
    "y" : c
}

df2 = pd.DataFrame(data2)

unformattedDF = pd.DataFrame(unformatted)

newdf = new._append(unformattedDF, ignore_index=False)

#appending the electricity maps data onto the region's datasheet
write_df_to_sheets(newdf, "US-NY-NYIS")

lows = df.sort_values(by="Carbon", ascending=True)

data = pd.DataFrame()

write_df_to_sheets(df2, "yesterdayNYIS")

In [ ]:
# US-NY-NYIS  forecasting
region = "US-NY-NYIS"
starting = 18062

import pandas as pd
from prophet import Prophet
import gspread
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.auth import default
from datetime import date
from datetime import datetime

def numOfDays(date1, date2):
  #check which date is greater to avoid days output in negative number
    if date2 > date1:
        return (date2-date1).days
    else:
        return (date1-date2).days

# Driver program
created = date(2025, 10, 25)
d = datetime.now().date()
diff = int(numOfDays(created, d))
gap = 24*diff


# Authenticate and create a client
creds, _ = default()
gc = gspread.authorize(creds)
auth.authenticate_user()

#reads the region's CI history
data = gc.open(region).sheet1
d = data.get_all_values()
data = pd.DataFrame(d, columns=["ds","y"])
df=data.drop(index=0)

#fits the ML model on the data
m = Prophet(changepoint_prior_scale=0.01).fit(df)
future = m.make_future_dataframe(periods=24, freq='H')
fcst = m.predict(future)

# isolates the last 24 values: the next 24 hrs of CI
x24 = fcst.tail(24)
ascending = x24.sort_values(by=["yhat"], ascending=True)
filter = ["ds","yhat"]
almost=ascending[filter]
almost["string"] = almost["ds"].astype("string")

fil = ["string", "yhat"]
final = almost[fil]
ok = final.head(4)
sum=0
print(x24)

dfx_listY = []
dfx_listDS = []

# calculates avg CI for the day
for i in range(24):
  y=i+starting+gap
  x=x24.loc[y,"yhat"]
  dfx_listY.append(x)
  ds=x24.loc[y,"ds"]
  dfx_listDS.append(str(ds))
  sum+=x

avg = sum/24
print(avg)

dfx = pd.DataFrame({"ds":dfx_listDS, "y":dfx_listY})

def write_df_to_sheets(df, sheet_name):
  values = [df.columns.values.tolist()] + df.values.tolist()
  sheet = gc.open(sheet_name).sheet1
  sheet.update(values)

# writes the forecast to a google sheet
write_df_to_sheets(dfx, "forecastNYIS")

avDF = pd.DataFrame({"yhat":[avg]})

#writes the 4 low CI hours of tomorrow onto a google sheet, as well as the average CI
write_df_to_sheets(ok, "lowsNYIS")
write_df_to_sheets(avDF, "avgNYIS")

DEBUG:cmdstanpy:input tempfile: /tmp/tmpg56yps50/k1lia41p.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpg56yps50/qsyi3sbk.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.12/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=51769', 'data', 'file=/tmp/tmpg56yps50/k1lia41p.json', 'init=/tmp/tmpg56yps50/qsyi3sbk.json', 'output', 'file=/tmp/tmpg56yps50/prophet_modelgc2gyt3j/prophet_model-20251025011100.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
01:11:00 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
01:11:07 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
/usr/local/lib/python3.12/dist-packages/prophet/forecaster.py:1872: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  dates = pd.date_range(
/tmp/ipython-input-2404594330.py:49: SettingWithCop

                       ds       trend  yhat_lower  yhat_upper  trend_lower  \
18062 2025-10-25 02:00:00  327.693690  298.104404  342.265308   327.693690   
18063 2025-10-25 03:00:00  327.700275  293.674077  335.504165   327.700275   
18064 2025-10-25 04:00:00  327.706860  288.222604  331.476356   327.706860   
18065 2025-10-25 05:00:00  327.713446  283.719862  328.379961   327.713446   
18066 2025-10-25 06:00:00  327.720031  279.355991  324.735588   327.720031   
18067 2025-10-25 07:00:00  327.726616  277.660085  323.685246   327.726616   
18068 2025-10-25 08:00:00  327.733202  277.861982  322.662522   327.733202   
18069 2025-10-25 09:00:00  327.739787  279.088367  325.494462   327.739787   
18070 2025-10-25 10:00:00  327.746372  283.564583  328.696890   327.746372   
18071 2025-10-25 11:00:00  327.752957  288.777229  333.221740   327.752957   
18072 2025-10-25 12:00:00  327.759543  291.816500  334.662496   327.759543   
18073 2025-10-25 13:00:00  327.766128  291.283708  336.039675   

////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
                                    
                                    US-MIDW-MISO below

////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

In [ ]:
# US-MIDW-MISO  updating past data

import pandas as pd
from prophet import Prophet
import gspread
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.auth import default
import requests

# Authenticate and create a client
creds, _ = default()
gc = gspread.authorize(creds)

# API call: past 24 hours of carbon intensity data of energy region
response = requests.get(
    "https://api.electricitymap.org/v3/carbon-intensity/history?zone=US-MIDW-MISO",
    headers={
        "auth-token": token
    }
)


if response.status_code == 200:
    # If the request was successful, parse the data
    global histCI
    data = response.json()
    histCI = pd.DataFrame.from_dict(data['history'])
else:
    print(f'Failed to retrieve data: {response.status_code}')

update = gc.open("US-MIDW-MISO").sheet1
n = update.get_all_values()
new = pd.DataFrame(n, columns=["ds","y"])
z = pd.to_datetime(histCI["datetime"])
df = pd.DataFrame(z, columns=["datetime"])
df["new"]=df["datetime"].astype(str)
df["Carbon"] = histCI["carbonIntensity"]
n=df.drop("datetime", axis="columns")
n.rename(columns={"new":"ds"}, inplace=True)

def write_df_to_sheets(df, sheet_name):
  values = [df.columns.values.tolist()] + df.values.tolist()
  sheet = gc.open(sheet_name).sheet1
  sheet.update(values)

l=[]
t=[]
#reformatting data from electricity maps and updating the lists
p = n["ds"]
data = pd.DataFrame(p)
for i in range (24):
  s = str(data.loc[i,"ds"])
  formatted = s.replace("+00:00","")
  clean = formatted.split(" ")
  time = clean[1]
  final = time.split(":")
  final1 = final[0]
  l.append(final1)
  t.append(formatted)
c = n["Carbon"]
data2 = {
    "ds" : l,
    "y" : c
}

unformatted = {
    "ds" : t,
    "y" : c
}

df2 = pd.DataFrame(data2)

unformattedDF = pd.DataFrame(unformatted)

newdf = new._append(unformattedDF, ignore_index=False)

# appending the electricity maps data onto the region's datasheet
write_df_to_sheets(newdf, "US-MIDW-MISO")

lows = df.sort_values(by="Carbon", ascending=True)

data = pd.DataFrame()

write_df_to_sheets(df2, "yesterdayMISO")

In [ ]:
# US-MIDW-MISO  forecasting
region = "US-MIDW-MISO"
starting = 18048

import pandas as pd
from prophet import Prophet
import gspread
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.auth import default
from datetime import date
from datetime import datetime

def numOfDays(date1, date2):
  #check which date is greater to avoid days output in negative number
    if date2 > date1:
        return (date2-date1).days
    else:
        return (date1-date2).days

# Driver program
created = date(2025, 10, 25)
d = datetime.now().date()
diff = int(numOfDays(created, d))
gap = 24*diff

# Authenticate and create a client
creds, _ = default()
gc = gspread.authorize(creds)
auth.authenticate_user()

#reads the region's CI history
data = gc.open(region).sheet1
d = data.get_all_values()
data = pd.DataFrame(d, columns=["ds","y"])
df=data.drop(index=0)

#fits the ML model on the data
m = Prophet(changepoint_prior_scale=0.01).fit(df)
future = m.make_future_dataframe(periods=24, freq='H')
fcst = m.predict(future)

# isolates the last 24 values: the next 24 hrs of CI
x24 = fcst.tail(24)
ascending = x24.sort_values(by=["yhat"], ascending=True)
filter = ["ds","yhat"]
almost=ascending[filter]
almost["string"] = almost["ds"].astype("string")

fil = ["string", "yhat"]
final = almost[fil]
ok = final.head(4)
sum=0
print(x24)

dfx_listY = []
dfx_listDS = []

# calculates avg CI for the day
for i in range(24):
  y=i+starting + gap
  x=x24.loc[y,"yhat"]
  dfx_listY.append(x)
  ds=x24.loc[y,"ds"]
  dfx_listDS.append(str(ds))
  sum+=x

avg = sum/24
print(avg)

dfx = pd.DataFrame({"ds":dfx_listDS, "y":dfx_listY})

def write_df_to_sheets(df, sheet_name):
  values = [df.columns.values.tolist()] + df.values.tolist()
  sheet = gc.open(sheet_name).sheet1
  sheet.update(values)

# writes the forecast to a google sheet
write_df_to_sheets(dfx, "forecastMISO")

avDF = pd.DataFrame({"yhat":[avg]})

#writes the 4 low CI hours of tomorrow onto a google sheet, as well as the average CI
write_df_to_sheets(ok, "lowsMISO")
write_df_to_sheets(avDF, "avgMISO")

DEBUG:cmdstanpy:input tempfile: /tmp/tmpg56yps50/0ic39y0p.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpg56yps50/fp061h55.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.12/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=85703', 'data', 'file=/tmp/tmpg56yps50/0ic39y0p.json', 'init=/tmp/tmpg56yps50/fp061h55.json', 'output', 'file=/tmp/tmpg56yps50/prophet_modelnx4d7zxk/prophet_model-20251025011319.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
01:13:19 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
01:13:25 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
/usr/local/lib/python3.12/dist-packages/prophet/forecaster.py:1872: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  dates = pd.date_range(
/tmp/ipython-input-1758703362.py:48: SettingWithCop

                       ds       trend  yhat_lower  yhat_upper  trend_lower  \
18048 2025-10-25 02:00:00  517.476188  473.558926  580.979627   517.476188   
18049 2025-10-25 03:00:00  517.480685  468.749855  574.653826   517.480685   
18050 2025-10-25 04:00:00  517.485181  458.749279  566.981743   517.485181   
18051 2025-10-25 05:00:00  517.489678  447.001889  555.112901   517.489678   
18052 2025-10-25 06:00:00  517.494175  436.572308  546.494003   517.494175   
18053 2025-10-25 07:00:00  517.498672  439.826887  543.792045   517.498672   
18054 2025-10-25 08:00:00  517.503169  435.795771  541.255996   517.503169   
18055 2025-10-25 09:00:00  517.507666  433.848052  542.150386   517.507666   
18056 2025-10-25 10:00:00  517.512163  439.785334  547.045755   517.512163   
18057 2025-10-25 11:00:00  517.516660  451.020899  554.355063   517.516660   
18058 2025-10-25 12:00:00  517.521156  453.969310  561.826665   517.521156   
18059 2025-10-25 13:00:00  517.525653  459.228224  566.264100   

////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
                                    
                                    US-MIDA-PJM below

////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

In [ ]:
# US-MIDA-PJM  updating past data

import pandas as pd
from prophet import Prophet
import gspread
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.auth import default
import requests

# Authenticate and create a client
creds, _ = default()
gc = gspread.authorize(creds)

# API call: past 24 hours of carbon intensity data of energy region
response = requests.get(
    "https://api.electricitymap.org/v3/carbon-intensity/history?zone=US-MIDA-PJM",
    headers={
        "auth-token": token
    }
)


if response.status_code == 200:
    # If the request was successful, parse the data
    global histCI
    data = response.json()
    histCI = pd.DataFrame.from_dict(data['history'])
else:
    print(f'Failed to retrieve data: {response.status_code}')

update = gc.open("US-MIDA-PJM").sheet1
n = update.get_all_values()
new = pd.DataFrame(n, columns=["ds","y"])
z = pd.to_datetime(histCI["datetime"])
df = pd.DataFrame(z, columns=["datetime"])
df["new"]=df["datetime"].astype(str)
df["Carbon"] = histCI["carbonIntensity"]
n=df.drop("datetime", axis="columns")
n.rename(columns={"new":"ds"}, inplace=True)

def write_df_to_sheets(df, sheet_name):
  values = [df.columns.values.tolist()] + df.values.tolist()
  sheet = gc.open(sheet_name).sheet1
  sheet.update(values)

l=[]
t=[]
#reformatting data from electricity maps and updating the lists
p = n["ds"]
data = pd.DataFrame(p)
for i in range (24):
  s = str(data.loc[i,"ds"])
  formatted = s.replace("+00:00","")
  clean = formatted.split(" ")
  time = clean[1]
  final = time.split(":")
  final1 = final[0]
  l.append(final1)
  t.append(formatted)
c = n["Carbon"]
data2 = {
    "ds" : l,
    "y" : c
}

unformatted = {
    "ds" : t,
    "y" : c
}

df2 = pd.DataFrame(data2)

unformattedDF = pd.DataFrame(unformatted)

newdf = new._append(unformattedDF, ignore_index=False)

# appending the electricity maps data onto the region's datasheet
write_df_to_sheets(newdf, "US-MIDA-PJM")

lows = df.sort_values(by="Carbon", ascending=True)

data = pd.DataFrame()

write_df_to_sheets(df2, "yesterdayPJM")

In [ ]:
# US-MIDA-PJM  forecasting
region = "US-MIDA-PJM"
starting = 18024

import pandas as pd
from prophet import Prophet
import gspread
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.auth import default
from datetime import date
from datetime import datetime

def numOfDays(date1, date2):
  #check which date is greater to avoid days output in negative number
    if date2 > date1:
        return (date2-date1).days
    else:
        return (date1-date2).days

# Driver program
created = date(2025, 10, 24)
d = datetime.now().date()
diff = int(numOfDays(created, d))
gap = 24*diff


# Authenticate and create a client
creds, _ = default()
gc = gspread.authorize(creds)
auth.authenticate_user()

#reads the region's CI history
data = gc.open(region).sheet1
d = data.get_all_values()
data = pd.DataFrame(d, columns=["ds","y"])
df=data.drop(index=0)

#fits the ML model on the data
m = Prophet(changepoint_prior_scale=0.01).fit(df)
future = m.make_future_dataframe(periods=24, freq='H')
fcst = m.predict(future)

# isolates the last 24 values: the next 24 hrs of CI
x24 = fcst.tail(24)
ascending = x24.sort_values(by=["yhat"], ascending=True)
filter = ["ds","yhat"]
almost=ascending[filter]
almost["string"] = almost["ds"].astype("string")

fil = ["string", "yhat"]
final = almost[fil]
ok = final.head(4)
sum=0
print(x24)

dfx_listY = []
dfx_listDS = []

# calculates avg CI for the day
for i in range(24):
  y=i+starting + gap
  x=x24.loc[y,"yhat"]
  dfx_listY.append(x)
  ds=x24.loc[y,"ds"]
  dfx_listDS.append(str(ds))
  sum+=x

avg = sum/24
print(avg)

dfx = pd.DataFrame({"ds":dfx_listDS, "y":dfx_listY})

def write_df_to_sheets(df, sheet_name):
  values = [df.columns.values.tolist()] + df.values.tolist()
  sheet = gc.open(sheet_name).sheet1
  sheet.update(values)

# writes the forecast to a google sheet
write_df_to_sheets(dfx, "forecastPJM")

avDF = pd.DataFrame({"yhat":[avg]})

#writes the 4 low CI hours of tomorrow onto a google sheet, as well as the average CI
write_df_to_sheets(ok, "lowsPJM")
write_df_to_sheets(avDF, "avgPJM")

DEBUG:cmdstanpy:input tempfile: /tmp/tmpg56yps50/zm3vvhqf.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpg56yps50/0g6kd1gq.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.12/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=65051', 'data', 'file=/tmp/tmpg56yps50/zm3vvhqf.json', 'init=/tmp/tmpg56yps50/0g6kd1gq.json', 'output', 'file=/tmp/tmpg56yps50/prophet_model_9cwztbo/prophet_model-20251025011624.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
01:16:24 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
01:16:42 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
/usr/local/lib/python3.12/dist-packages/prophet/forecaster.py:1872: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  dates = pd.date_range(
/tmp/ipython-input-80402489.py:49: SettingWithCopyW

                       ds       trend  yhat_lower  yhat_upper  trend_lower  \
18048 2025-10-25 02:00:00  434.049816  389.082385  441.293509   434.049816   
18049 2025-10-25 03:00:00  434.056210  383.465610  437.897441   434.056210   
18050 2025-10-25 04:00:00  434.062604  377.863122  430.887096   434.062604   
18051 2025-10-25 05:00:00  434.068997  367.581477  424.475162   434.068997   
18052 2025-10-25 06:00:00  434.075391  365.377280  419.660843   434.075391   
18053 2025-10-25 07:00:00  434.081785  363.045149  416.591646   434.081785   
18054 2025-10-25 08:00:00  434.088179  363.260381  417.343719   434.088179   
18055 2025-10-25 09:00:00  434.094573  366.995174  420.803088   434.094573   
18056 2025-10-25 10:00:00  434.100966  367.053533  422.994923   434.100966   
18057 2025-10-25 11:00:00  434.107360  371.044832  423.492270   434.107360   
18058 2025-10-25 12:00:00  434.113754  370.309477  423.637991   434.113754   
18059 2025-10-25 13:00:00  434.120148  371.333279  427.797143   

////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
                                    
                                    US-NE-ISNE below

////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

In [ ]:
# US-NE-ISNE  updating past data

import pandas as pd
from prophet import Prophet
import gspread
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.auth import default
import requests

# Authenticate and create a client
creds, _ = default()
gc = gspread.authorize(creds)

# API call: past 24 hours of carbon intensity data of energy region
response = requests.get(
    "https://api.electricitymap.org/v3/carbon-intensity/history?zone=US-NE-ISNE",
    headers={
        "auth-token": token
    }
)


if response.status_code == 200:
    # If the request was successful, parse the data
    global histCI
    data = response.json()
    histCI = pd.DataFrame.from_dict(data['history'])
else:
    print(f'Failed to retrieve data: {response.status_code}')

update = gc.open("US-NE-ISNE").sheet1
n = update.get_all_values()
new = pd.DataFrame(n, columns=["ds","y"])
z = pd.to_datetime(histCI["datetime"])
df = pd.DataFrame(z, columns=["datetime"])
df["new"]=df["datetime"].astype(str)
df["Carbon"] = histCI["carbonIntensity"]
n=df.drop("datetime", axis="columns")
n.rename(columns={"new":"ds"}, inplace=True)

def write_df_to_sheets(df, sheet_name):
  values = [df.columns.values.tolist()] + df.values.tolist()
  sheet = gc.open(sheet_name).sheet1
  sheet.update(values)

l=[]
t=[]
#reformatting data from electricity maps and updating the lists
p = n["ds"]
data = pd.DataFrame(p)
for i in range (24):
  s = str(data.loc[i,"ds"])
  formatted = s.replace("+00:00","")
  clean = formatted.split(" ")
  time = clean[1]
  final = time.split(":")
  final1 = final[0]
  l.append(final1)
  t.append(formatted)
c = n["Carbon"]
data2 = {
    "ds" : l,
    "y" : c
}

unformatted = {
    "ds" : t,
    "y" : c
}

df2 = pd.DataFrame(data2)

unformattedDF = pd.DataFrame(unformatted)

newdf = new._append(unformattedDF, ignore_index=False)

# appending the electricity maps data onto the region's datasheet
write_df_to_sheets(newdf, "US-NE-ISNE")

lows = df.sort_values(by="Carbon", ascending=True)

data = pd.DataFrame()

write_df_to_sheets(df2, "yesterdayISNE")

In [ ]:
# US-NE-ISNE forecasting
region = "US-NE-ISNE"
starting = 9287

import pandas as pd
from prophet import Prophet
import gspread
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.auth import default
from datetime import date
from datetime import datetime

def numOfDays(date1, date2):
  #check which date is greater to avoid days output in negative number
    if date2 > date1:
        return (date2-date1).days
    else:
        return (date1-date2).days

# Driver program
created = date(2025, 10, 25)
d = datetime.now().date()
diff = int(numOfDays(created, d))
gap = 24*diff


# Authenticate and create a client
creds, _ = default()
gc = gspread.authorize(creds)
auth.authenticate_user()

#reads the region's CI history
data = gc.open(region).sheet1
d = data.get_all_values()
data = pd.DataFrame(d, columns=["ds","y"])
df=data.drop(index=0)

#fits the ML model on the data
m = Prophet(changepoint_prior_scale=0.01).fit(df)
future = m.make_future_dataframe(periods=24, freq='H')
fcst = m.predict(future)

# isolates the last 24 values: the next 24 hrs of CI
x24 = fcst.tail(24)
ascending = x24.sort_values(by=["yhat"], ascending=True)
filter = ["ds","yhat"]
almost=ascending[filter]
almost["string"] = almost["ds"].astype("string")

fil = ["string", "yhat"]
final = almost[fil]
ok = final.head(4)
sum=0
print(x24)

dfx_listY = []
dfx_listDS = []

# calculates avg CI for the day
for i in range(24):
  y=i+starting+gap
  x=x24.loc[y,"yhat"]
  dfx_listY.append(x)
  ds=x24.loc[y,"ds"]
  dfx_listDS.append(str(ds))
  sum+=x

avg = sum/24
print(avg)

dfx = pd.DataFrame({"ds":dfx_listDS, "y":dfx_listY})

def write_df_to_sheets(df, sheet_name):
  values = [df.columns.values.tolist()] + df.values.tolist()
  sheet = gc.open(sheet_name).sheet1
  sheet.update(values)

# writes the forecast to a google sheet
write_df_to_sheets(dfx, "forecastISNE")

avDF = pd.DataFrame({"yhat":[avg]})

#writes the 4 low CI hours of tomorrow onto a google sheet, as well as the average CI
write_df_to_sheets(ok, "lowsISNE")
write_df_to_sheets(avDF, "avgISNE")

DEBUG:cmdstanpy:input tempfile: /tmp/tmpg56yps50/hvxmk1na.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpg56yps50/3oy7wyki.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.12/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=26321', 'data', 'file=/tmp/tmpg56yps50/hvxmk1na.json', 'init=/tmp/tmpg56yps50/3oy7wyki.json', 'output', 'file=/tmp/tmpg56yps50/prophet_modelqhihwfab/prophet_model-20251025012238.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
01:22:38 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
01:22:40 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
/usr/local/lib/python3.12/dist-packages/prophet/forecaster.py:1872: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  dates = pd.date_range(
/tmp/ipython-input-1731140904.py:49: SettingWithCop

                      ds       trend  yhat_lower  yhat_upper  trend_lower  \
9287 2025-10-25 02:00:00  335.545242  325.649584  386.801108   335.545242   
9288 2025-10-25 03:00:00  335.552533  318.816992  379.463461   335.552533   
9289 2025-10-25 04:00:00  335.559823  315.753951  373.978520   335.559823   
9290 2025-10-25 05:00:00  335.567113  307.462496  364.100512   335.567113   
9291 2025-10-25 06:00:00  335.574403  303.782731  363.690958   335.574403   
9292 2025-10-25 07:00:00  335.581694  304.402489  363.564149   335.581694   
9293 2025-10-25 08:00:00  335.588984  305.917142  364.018451   335.588984   
9294 2025-10-25 09:00:00  335.596274  306.621790  366.597281   335.596274   
9295 2025-10-25 10:00:00  335.603565  310.391968  371.772524   335.603565   
9296 2025-10-25 11:00:00  335.610855  314.217099  376.159449   335.610855   
9297 2025-10-25 12:00:00  335.618145  315.595635  375.657386   335.618145   
9298 2025-10-25 13:00:00  335.625435  318.198465  377.066714   335.625435   

////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
                                    
                                    US-TEX-ERCO below

////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

In [ ]:
# US-TEX-ERCO  updating past data

import pandas as pd
from prophet import Prophet
import gspread
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.auth import default
import requests

# Authenticate and create a client
creds, _ = default()
gc = gspread.authorize(creds)

# API call: past 24 hours of carbon intensity data of energy region
response = requests.get(
    "https://api.electricitymap.org/v3/carbon-intensity/history?zone=US-TEX-ERCO",
    headers={
        "auth-token": token
    }
)


if response.status_code == 200:
    # If the request was successful, parse the data
    global histCI
    data = response.json()
    histCI = pd.DataFrame.from_dict(data['history'])
else:
    print(f'Failed to retrieve data: {response.status_code}')

update = gc.open("US-TEX-ERCO").sheet1
n = update.get_all_values()
new = pd.DataFrame(n, columns=["ds","y"])
z = pd.to_datetime(histCI["datetime"])
df = pd.DataFrame(z, columns=["datetime"])
df["new"]=df["datetime"].astype(str)
df["Carbon"] = histCI["carbonIntensity"]
n=df.drop("datetime", axis="columns")
n.rename(columns={"new":"ds"}, inplace=True)

def write_df_to_sheets(df, sheet_name):
  values = [df.columns.values.tolist()] + df.values.tolist()
  sheet = gc.open(sheet_name).sheet1
  sheet.update(values)

l=[]
t=[]
#reformatting data from electricity maps and updating the lists
p = n["ds"]
data = pd.DataFrame(p)
for i in range (24):
  s = str(data.loc[i,"ds"])
  formatted = s.replace("+00:00","")
  clean = formatted.split(" ")
  time = clean[1]
  final = time.split(":")
  final1 = final[0]
  l.append(final1)
  t.append(formatted)
c = n["Carbon"]
data2 = {
    "ds" : l,
    "y" : c
}

unformatted = {
    "ds" : t,
    "y" : c
}

df2 = pd.DataFrame(data2)

unformattedDF = pd.DataFrame(unformatted)

newdf = new._append(unformattedDF, ignore_index=False)

# appending the electricity maps data onto the region's datasheet
write_df_to_sheets(newdf, "US-TEX-ERCO")

lows = df.sort_values(by="Carbon", ascending=True)

data = pd.DataFrame()

write_df_to_sheets(df2, "yesterdayTEX")

In [ ]:
# US-TEX-ERCO  forecasting
region = "US-TEX-ERCO"
starting = 9288

import pandas as pd
from prophet import Prophet
import gspread
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.auth import default
from datetime import date
from datetime import datetime

def numOfDays(date1, date2):
  #check which date is greater to avoid days output in negative number
    if date2 > date1:
        return (date2-date1).days
    else:
        return (date1-date2).days

# Driver program
created = date(2025, 10, 25)
d = datetime.now().date()
diff = int(numOfDays(created, d))
gap = 24*diff


# Authenticate and create a client
creds, _ = default()
gc = gspread.authorize(creds)
auth.authenticate_user()

#reads the region's CI history
data = gc.open(region).sheet1
d = data.get_all_values()
data = pd.DataFrame(d, columns=["ds","y"])
df=data.drop(index=0)

#fits the ML model on the data
m = Prophet(changepoint_prior_scale=0.01).fit(df)
future = m.make_future_dataframe(periods=24, freq='H')
fcst = m.predict(future)

# isolates the last 24 values: the next 24 hrs of CI
x24 = fcst.tail(24)
ascending = x24.sort_values(by=["yhat"], ascending=True)
filter = ["ds","yhat"]
almost=ascending[filter]
almost["string"] = almost["ds"].astype("string")

fil = ["string", "yhat"]
final = almost[fil]
ok = final.head(4)
sum=0
print(x24)

dfx_listY = []
dfx_listDS = []

# calculates avg CI for the day
for i in range(24):
  y=i+starting + gap
  x=x24.loc[y,"yhat"]
  dfx_listY.append(x)
  ds=x24.loc[y,"ds"]
  dfx_listDS.append(str(ds))
  sum+=x

avg = sum/24
print(avg)

dfx = pd.DataFrame({"ds":dfx_listDS, "y":dfx_listY})

def write_df_to_sheets(df, sheet_name):
  values = [df.columns.values.tolist()] + df.values.tolist()
  sheet = gc.open(sheet_name).sheet1
  sheet.update(values)

# writes the forecast to a google sheet
write_df_to_sheets(dfx, "forecastTEX")

avDF = pd.DataFrame({"yhat":[avg]})

#writes the 4 low CI hours of tomorrow onto a google sheet, as well as the average CI
write_df_to_sheets(ok, "lowsTEX")
write_df_to_sheets(avDF, "avgTEX")

DEBUG:cmdstanpy:input tempfile: /tmp/tmpg56yps50/6p2zvb37.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpg56yps50/a9xyican.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.12/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=66202', 'data', 'file=/tmp/tmpg56yps50/6p2zvb37.json', 'init=/tmp/tmpg56yps50/a9xyican.json', 'output', 'file=/tmp/tmpg56yps50/prophet_modelcg_sxnss/prophet_model-20251025012415.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
01:24:15 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
01:24:15 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
/usr/local/lib/python3.12/dist-packages/prophet/forecaster.py:1872: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  dates = pd.date_range(
/tmp/ipython-input-320546910.py:49: SettingWithCopy

                      ds       trend  yhat_lower  yhat_upper  trend_lower  \
9288 2025-10-25 02:00:00  355.989876  293.639011  438.311167   355.989876   
9289 2025-10-25 03:00:00  355.990490  279.678357  429.773061   355.990490   
9290 2025-10-25 04:00:00  355.991103  272.097135  420.384566   355.991103   
9291 2025-10-25 05:00:00  355.991716  255.086520  404.700772   355.991716   
9292 2025-10-25 06:00:00  355.992330  248.839680  398.763029   355.992330   
9293 2025-10-25 07:00:00  355.992943  239.495652  394.544473   355.992943   
9294 2025-10-25 08:00:00  355.993557  239.900724  389.411316   355.993557   
9295 2025-10-25 09:00:00  355.994170  254.778477  403.964621   355.994170   
9296 2025-10-25 10:00:00  355.994784  258.896335  406.372045   355.994784   
9297 2025-10-25 11:00:00  355.995397  269.089195  421.327913   355.995397   
9298 2025-10-25 12:00:00  355.996011  270.417374  411.453330   355.996011   
9299 2025-10-25 13:00:00  355.996624  255.955168  403.412735   355.996624   